In [1]:
%load_ext autoreload
%autoreload 2
%aimport -jax
%aimport -jaxlib

In [2]:
import jax
import jax.numpy as jnp
from stanza.envs.linear import LinearSystem
import stanza.policies as policies
from jax.random import PRNGKey
from stanza.util.random import PRNGSequence


In [7]:
x = jax.lax.while_loop(cond_fun = lambda x: False, 
                   body_fun = lambda x: x+1,init_val = 0)
print(jnp.arange(1))

SyntaxError: unmatched ')' (2313576938.py, line 3)

In [100]:
from stanza.dataclasses import dataclass
from stanza.policies import PolicyOutput


def create_integrator_matrices(dim: int):
    A = jnp.zeros((dim,dim))
    B = jnp.zeros((dim,1))
    for i in range(dim-1):
        A = A.at[i,i+1].set(1)
    B = B.at[dim-1,0].set(1)
    return A, B

def create_integrator_environment(dim: int, x0_min=-1, x0_max=1,
                xmax=1, xmin=-1,
                umax=1, umin=1):
    Q = jnp.eye(dim)
    R = jnp.eye(1)
    A, B = create_integrator_matrices(dim)
    return LinearSystem(A=A,B=B,Q=Q,R=R,x0_min=x0_min,
                        x0_max=x0_max,xmax=xmax,umin=umin,
                        xmin=xmin,
                        umax=umax)
@dataclass(jax=True)
class Uniform1dPolicy():
    umin : int = -1
    umax : int = 1

    def __call__(self, input):
        print(input)
        action = jax.random.uniform(input.rng_key, (1,), jnp.float32, self.umin, self.umax) 
        return PolicyOutput(action=action)
    



In [ ]:
from stanza.data import Data
from stanza.data.trajectory import Timestep
import stanza.envs as envs
from stanza.goal_conditioned.rollin.curriculum_utils import make_gaussian_noiser_scalar_variance
from stanza.goal_conditioned.rollin.rollin_sampler import RollInSampler, UniformRollinHelper

from stanza.policies.mpc import MPC
from stanza.solver.ilqr import iLQRSolver
from stanza.util.logging import logger

logger.info("Creating environment")

horizon = 50

solver_t = iLQRSolver()
expert_policy=MPC(
            # Sample action
            action_sample=env.sample_action(PRNGKey(0)),
            cost_fn=env.cost, 
            model_fn=env.step,
            horizon_length=horizon,
            solver=solver_t,
            receed=False
        )

def rollout_policy(rng_key, pol,env,horizon):
    # random init angle and angular velocity
    key1,key2,key3 = jax.random.split(rng_key,3)
    x_0 = env.reset(key1) 
    roll = policies.rollout(model = env.step,
                     state0 = x_0,
                     policy = pol,
                     length = horizon,
                     last_state = False,
                     policy_rng_key=key3,
                     model_rng_key=key2)
    
    return Data.from_pytree(Timestep(roll.states,roll.actions))

def batch_roll(rng_key, num_t, pol,env,horizon):
    roll_func = jax.vmap(rollout_policy,in_axes=(0,None,None,None))
    rng_keys = jax.random.split(rng_key,num_t)
    print(rng_keys)
    return Data.from_pytree(roll_func(rng_keys,pol,env,horizon))

num_t = 5
horizon = 10

env = create_integrator_environment(dim = 4,
                                    x0_max=1,
                                    x0_min=1)
my_key = PRNGKey(42)

my_pol = Uniform1dPolicy(umin=2,umax=2)
roll = batch_roll(my_key,pol=my_pol,env=env,
                  horizon=horizon,num_t=1)
data = roll
print(data)
rollin_helper = UniformRollinHelper()
# fix ths
action = Data.from_pytree(jnp.ones(1,))
gaussian_noiser = make_gaussian_noiser_scalar_variance(sigma = 1)
print(gaussian_noiser(rng=my_key,value=action))

gaussian_noiser = make_gaussian_noiser_scalar_variance(sigma = .99)
print(gaussian_noiser(rng=my_key,value=action))

#print(gaussian_noiser(rng=my_key,value=jnp.ones(1,)))




In [ ]:

sig1,sig2 = 1. ,.5

rollin_helper = UniformRollinHelper(delta_t_min=1,
                                    delta_t_max = 3,
                                    roll_len_min = 2,
                                    roll_len_max = 3,
                                    min_start_t = 0)

#rollin_helper = UniformRollinHelper()
def make_sampler(sigma, full_goal_state_action = True):
    gaussian_noiser = make_gaussian_noiser_scalar_variance(sigma=sigma)
    sampler = RollInSampler(env=env, traj_data=data,rollin_helper=rollin_helper,
                   action_noiser=gaussian_noiser)
    if full_goal_state_action:
        sampler_fn = (lambda key: sampler.sample_goal_state_action(key))
    else:
        sampler_fn = (lambda key: sampler.sample_gc_state(key))
    return sampler_fn

def print_sample(start_state, goal, start_action, info):
    print("start_state")
    print(start_state)
    print("goal")
    print(goal)
    print("start_action")
    print(start_action)
    print("info")
    print(info)

rng = PRNGSequence(PRNGKey(4))
next(rng)
rng2 = PRNGSequence(next(rng))
key2 = next(rng2)
print('delkey', key2)
delta_t = jax.random.randint(key2, (), minval = 1,
              
                               maxval = 4)
print(delta_t)
sigmas = [1,.9]
key_nums = [4]
for i in key_nums:
    print(' ')
    print('key num', i)
    key = PRNGKey(i)
    for x in sigmas:
        sampler =  make_sampler(x) 
        print_sample(*sampler(key))



In [209]:
x = [1,2,3]
print(type(jnp.array(x)))
y = None
if y is None or y.hello() or False:
    print('hi')


<class 'jaxlib.xla_extension.ArrayImpl'>
hi


In [208]:
### create mixture environment
## this works


from stanza.envs.goal_conditioned_envs.gc_pendulum import make_gc_pendulum_env
from stanza.goal_conditioned import GCEnvironment
from stanza.goal_conditioned.rollin.curriculum_utils import MixtureEnvironment


envs_list = []
sigmas = [0,.1,.5,.9,1.] 
probs_list = jnp.array([1 for i in range(len(sigmas))])


def make_gc_state_sampler(sigma):
    gaussian_noiser = make_gaussian_noiser_scalar_variance(sigma=sigma)
    sampler = RollInSampler(env=env, traj_data=data,rollin_helper=rollin_helper,
                   action_noiser=gaussian_noiser)
    return (lambda key: sampler.sample_gc_state(key))

for sigma in sigmas:
    # gets a gc_state sampler from a given sigma
    sampler = make_gc_state_sampler(sigma)
    gc_env = GCEnvironment(env=env, gs_sampler=sampler)
    envs_list.append(gc_env)

# creates a mixture of environments in envs_list
# based on probs_list 
# here, it is using the first environment as the base env 
mix_env_1 = MixtureEnvironment(envs_list=envs_list, probs_list=probs_list, 
                             mixture_base_env=None,first_is_base=True)

base_env = envs_list[0]
# check this is the same
mix_env_2 = MixtureEnvironment(envs_list=envs_list, probs_list=probs_list, 
                             mixture_base_env=mix_env_1)

# check this is the same
# default mixture is uniform
mix_env_3 = MixtureEnvironment(envs_list=envs_list, 
                             mixture_base_env=mix_env_1)

key = PRNGKey(42)
key_seqs = [PRNGSequence(key), PRNGSequence(key), PRNGSequence(key)]

mixture_envs = [mix_env_1, mix_env_2, mix_env_3]

env_ind_dicts = [{}, {}, {}]
for i in range(20):
    for j, mix_env in enumerate(mixture_envs):
        env, key2, env_ind = mix_env_1.sample_env(next(key_seqs[j]))
        env_ind_dicts[j][i] = env_ind

print(env_ind_dicts[0])
print(env_ind_dicts[1])
print(env_ind_dicts[2])




{0: Array(4, dtype=int32), 1: Array(2, dtype=int32), 2: Array(2, dtype=int32), 3: Array(2, dtype=int32), 4: Array(1, dtype=int32), 5: Array(1, dtype=int32), 6: Array(4, dtype=int32), 7: Array(4, dtype=int32), 8: Array(1, dtype=int32), 9: Array(3, dtype=int32), 10: Array(1, dtype=int32), 11: Array(1, dtype=int32), 12: Array(4, dtype=int32), 13: Array(3, dtype=int32), 14: Array(0, dtype=int32), 15: Array(2, dtype=int32), 16: Array(4, dtype=int32), 17: Array(4, dtype=int32), 18: Array(0, dtype=int32), 19: Array(3, dtype=int32)}
{0: Array(4, dtype=int32), 1: Array(2, dtype=int32), 2: Array(2, dtype=int32), 3: Array(2, dtype=int32), 4: Array(1, dtype=int32), 5: Array(1, dtype=int32), 6: Array(4, dtype=int32), 7: Array(4, dtype=int32), 8: Array(1, dtype=int32), 9: Array(3, dtype=int32), 10: Array(1, dtype=int32), 11: Array(1, dtype=int32), 12: Array(4, dtype=int32), 13: Array(3, dtype=int32), 14: Array(0, dtype=int32), 15: Array(2, dtype=int32), 16: Array(4, dtype=int32), 17: Array(4, dtype=

In [ ]:
env_maker = (lambda envir, sampler: 
             GCEnvironment(env=envir, gs_sampler=sampler))         
base_env = env

#start by testing none_getters

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'